<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 5px; height: 50px"> 

#   Personalizing Music Video Recommendations with Emotional Intelligence

> Capstone Project: Lionel Foo

---

### <b> Notebook: 03B Scraping Youtube Comments</b>

#### Overview

* Scraping 1000 comments each from each of the 100 youtube music videos for recommender system.


---

### 01 Import Libraries

In [1]:
#!pip install google-api-python-client
# Imports
import googleapiclient.discovery
import pandas as pd
import contractions
import re

---

### 02 Data Collection - Scraping Youtube

In [2]:
# authenticate with Google API - redacted key in API key as required
dev = ""

In [4]:
# Retrieve up to 1000 comments for a YouTube video using the YouTube Data API v3
# Parameters: video - YouTube video ID
# Returns a DataFrame with comment details including author, timestamp, likes, text, video ID, and public status
api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = dev

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

def getcomments(video):
  request = youtube.commentThreads().list(
      part="snippet",
      videoId=video,
      maxResults=100
  )

  comments = []
  count = 0  # Add a counter

  # Execute the request.
  response = request.execute()

  # Get the comments from the response.
  for item in response['items']:
      comment = item['snippet']['topLevelComment']['snippet']
      public = item['snippet']['isPublic']
      comments.append([
          comment['authorDisplayName'],
          comment['publishedAt'],
          comment['likeCount'],
          comment['textOriginal'],
          comment['videoId'],
          public
      ])
      count += 1  # Increase the counter by 1
      if count >= 1000:  # Check if the counter has reached 1000
          break

  while count < 1000:  # Check if the counter is less than 1000
    try:
     nextPageToken = response['nextPageToken']
    except KeyError:
     break
    nextPageToken = response['nextPageToken']
    # Create a new request object with the next page token.
    nextRequest = youtube.commentThreads().list(part="snippet", videoId=video, maxResults=100, pageToken=nextPageToken)
    # Execute the next request.
    response = nextRequest.execute()
    # Get the comments from the next response.
    for item in response['items']:
      comment = item['snippet']['topLevelComment']['snippet']
      public = item['snippet']['isPublic']
      comments.append([
          comment['authorDisplayName'],
          comment['publishedAt'],
          comment['likeCount'],
          comment['textOriginal'],
          comment['videoId'],
          public
      ])
      count += 1  # Increment the counter
      if count >= 1000:  # Check if the counter has reached 500
          break

  df2 = pd.DataFrame(comments, columns=['author', 'updated_at', 'like_count', 'text','video_id','public'])
  return df2

In [5]:
# Concatenate comments from multiple YouTube videos into a single DataFrame
df = pd.DataFrame()
for i in ['G7KNmW9a75Y','Oa_RSwwpPaA', '8UMnAIaBofo', 'Iq8h3GEe22o', 'H5v3kku4y6Q', 'suAR1PYFNYA', 'wAc-5m-wNac',
              'm4_9TFeMfJE', 'UTmZqY_arzk', 'MoN9ql6Yymw', 'eB6txyhHFG4', 'r1AzafxuijY', 'UNo0TG9LwwI', 'je0roKRn3nY', 'cW8VLC9nnTo',
                'QU9c0053UAU', 'YjCtucn29JQ',  'LDY_XyxBu8A', 'OiC1rgCPmUQ', 'mNEUkkoUoIA', 'wZTIdnooV-s', '90RLzVUuXe4', 'ccu6JuC21rk',
                  'ZDZiXmCl4pk', '2JEv8AVW8vw', 'crtQSTYWtqE', 'PEM0Vs8jf1w', 'MSRcC626prw', 'b1kbLwvqugk', 'kLbn61Z4LDI',
                    'RlPNh_PBZb4', 'OqEc_169ywY',  'Uq9gPaIzbe8', 'b3vJfR81xO0', 'Dll6VJ2C7wo', 'TO-_3tck2tg', 'YV8P4aEXzMU', 'O2CIAKVTOrc',
                      'a7GITgqwDVg', 'IZ3XMOdOdKM', 'XzOvgu3GPwY', 'YXt0Nw8xWh0', 'VF-FGf_ZZiI', '7aekxC_monc', 'Y2QpQP8wPG8',
                        'tu4HfcmMn1E', 'gVqGKkm7xBE', '7x5lqqji9ww', 'mHNCM-YALSA', '5AgDZcM8In8', 'YcSP1ZUf1eQ', 'QCtEe-zsCtQ', 'b7QlX3yR2xs',
                          'me19SUmWu2s', 'dCrQeUaXkLo', 'Opxhh9Oh3rg', 'Te3_VlimRw0', 'h8DLofLM7No', 'gPCCYMeXin0', 'YBGtzfK5Bak',
                          'u6wOyMUs74I', 'Mx_OexsUI2M', '9coyY-SPIXU', 'U3ASj1L6_sY', 'WMweEpGlu_U', 'kTJczUoc26U', 'mRD0-GxqHVo', 'niG3YMU6jFk', 'tQ0yjYUFKAE', '6swmTBVI83k',
               'qod03PVTLqk', 'GrAchTdepsU', 'nCg3ufihKyU', 'adLGHcj_fmA', 'LWeiydKl0mU', 'XXYlFuWEuKI', 'KIK3azN4w34', 'IzPQ_jA00bk', '3c909oqLfao', '4D89Qr5vH6U',
               '27nRqoBQLl4', 'bUjPPBxbQrQ', 'DAOZJPquY_w', 'U6n2NcJ7rLc', 'UTHLKHL_whs', 'D9G1VOjN_84', 'gNi_6U5Pm_o', 'Q1Yy0tNWtnE', 'gdZLi9oWNZg', 'D2KE2a5qo0g',
               '23g5HBOg3Ic', 'iqYK79jCssA', 'bLzUmfLckEw', '4VaqA-5aQTM', 'gXl0zgUhsAU', 'uS_y_65CcpA', 'QBPE2fZsVYU', 'JKrDdsgXuso', 'X0lRcz_IuPQ', 'vhumOLNSSJY']:
  df2 = getcomments(i)
  df = pd.concat([df, df2])

In [6]:
# Check summary of scraped dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97914 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   author      97914 non-null  object
 1   updated_at  97914 non-null  object
 2   like_count  97914 non-null  int64 
 3   text        97914 non-null  object
 4   video_id    97914 non-null  object
 5   public      97914 non-null  bool  
dtypes: bool(1), int64(1), object(4)
memory usage: 4.6+ MB


In [7]:
# reset index of dataframe
df = df.reset_index(drop=True)

In [8]:
# check rows of dataframe
df

,author,updated_at,like_count,text,video_id,public
0,@MileyCyrus,2023-01-13T00:39:35Z,158633,I can love me better than you can…. FLOWERS SI...,G7KNmW9a75Y,True
1,@adanilsoncapel,2024-02-24T09:15:02Z,0,I can buy myself flowers,G7KNmW9a75Y,True
2,@adanilsoncapel,2024-02-24T09:14:47Z,0,We were good we were gold,G7KNmW9a75Y,True
3,@Royalchess1,2024-02-24T09:02:54Z,0,"Ms. Miley,\n I have alot of mixed emotions w...",G7KNmW9a75Y,True
4,@mikerooney7600,2024-02-24T09:01:41Z,0,I love this song! It makes every day better,G7KNmW9a75Y,True
...,...,...,...,...,...,...
97909,@18t84,2023-09-24T18:15:08Z,2,❤,vhumOLNSSJY,True
97910,@69jdwalt,2023-09-24T11:36:02Z,1,Help... both needs it.,vhumOLNSSJY,True
97911,@69jdwalt,2023-09-24T10:30:15Z,0,No one should be taken over after.\nThis is a...,vhumOLNSSJY,True
97912,@ZShorts-gf7tz,2023-09-23T21:02:52Z,4,Absolutely amazing song. NF is not just talent...,vhumOLNSSJY,True


In [9]:
# check number of unique video ids
df['video_id'].nunique()

100

In [10]:
# Display the count of comments for each unique 'video_id'
df['video_id'].value_counts()

video_id
G7KNmW9a75Y    1000
adLGHcj_fmA    1000
GrAchTdepsU    1000
qod03PVTLqk    1000
6swmTBVI83k    1000
               ... 
vhumOLNSSJY    1000
3c909oqLfao     920
YV8P4aEXzMU     527
X0lRcz_IuPQ     435
wAc-5m-wNac      32
Name: count, Length: 100, dtype: int64

In [11]:
# export data frame lyrics as csv:
df.to_csv('Data/youtube_comments.csv', index=False)


---

### 03 EDA & Cleaning

In [12]:
# Display all data rows:
pd.set_option('display.max_rows', None)
pd.options.display.max_colwidth = 4000
df.head(100)

,author,updated_at,like_count,text,video_id,public
0,@MileyCyrus,2023-01-13T00:39:35Z,158633,I can love me better than you can…. FLOWERS SINGLE + VIDEO OUT NOW. \n\nhttps://mileycyrus.lnk.to/Flowers,G7KNmW9a75Y,True
1,@adanilsoncapel,2024-02-24T09:15:02Z,0,I can buy myself flowers,G7KNmW9a75Y,True
2,@adanilsoncapel,2024-02-24T09:14:47Z,0,We were good we were gold,G7KNmW9a75Y,True
3,@Royalchess1,2024-02-24T09:02:54Z,0,"Ms. Miley,\n I have alot of mixed emotions while watching your video. I could use many explanitives that you've heard before, but the words, the music and the video is something I've NEVER WITNESSED BEFORE. I MUST ADMIT, I've NEVER watched one of your videos before, but if I did, it's been awhile ago. I can tell you are in great shape, and I mean that in all honesty. Personally, I've been FORCED to suspend my VASA spa contract because of COVID, and I can't go mountain-biking (here in Utah) or play Olympic Table Tennis either because I have some operations to deal with, so I can appreciate you being in shape.\n I've had/have an idea for you, and only YOU can do this. I would like to see you make a song/video which would be an answer to your dad's song ACHEY BREAKY HEART. If I were your dad, I would be TOUCHED! I don't know if you've had this in mind before, but I've had it in my mind just recently. What Say You?! And a VERY WARM CONGRADULATION for winning your very FIRST GRAMMY!",G7KNmW9a75Y,True
4,@mikerooney7600,2024-02-24T09:01:41Z,0,I love this song! It makes every day better,G7KNmW9a75Y,True
5,@Ganesh-zs1iv,2024-02-24T08:07:13Z,0,It's Feb 2024 and still I am watching it,G7KNmW9a75Y,True
6,@ogtdkk5932,2024-02-24T07:06:47Z,0,"Si hay que idealizar alguna canción, esta es la correcta.....",G7KNmW9a75Y,True
7,@dabrown3973,2024-02-24T06:27:11Z,0,"❤❤❤😅😊😊😊😊🎉 OMG. She is the Queen. She followed me on Twitter. I can't believe. Everyone got their names, the swiftys, barbs,. I Got Miley Cyrus new name. THE MILEY POPS ♥. Let's spread that name Pops. We are are MILEY Pops. I'm first to give her the prprincess of pop",G7KNmW9a75Y,True
8,@user-np8gh6gc3s,2024-02-24T06:17:26Z,0,❤,G7KNmW9a75Y,True
9,@Marc.1776.,2024-02-24T05:40:39Z,0,Bruno mars did it better,G7KNmW9a75Y,True


In [13]:
# Check for data in column 'text' that contain line or tab characters:
print(df['text'].str.contains(r'\r|\t|\n').value_counts())
df[df['text'].str.contains(r'\r|\t|\n') == True].head()

text
False    92332
True      5582
Name: count, dtype: int64


,author,updated_at,like_count,text,video_id,public
0,@MileyCyrus,2023-01-13T00:39:35Z,158633,I can love me better than you can…. FLOWERS SINGLE + VIDEO OUT NOW. \n\nhttps://mileycyrus.lnk.to/Flowers,G7KNmW9a75Y,True
3,@Royalchess1,2024-02-24T09:02:54Z,0,"Ms. Miley,\n I have alot of mixed emotions while watching your video. I could use many explanitives that you've heard before, but the words, the music and the video is something I've NEVER WITNESSED BEFORE. I MUST ADMIT, I've NEVER watched one of your videos before, but if I did, it's been awhile ago. I can tell you are in great shape, and I mean that in all honesty. Personally, I've been FORCED to suspend my VASA spa contract because of COVID, and I can't go mountain-biking (here in Utah) or play Olympic Table Tennis either because I have some operations to deal with, so I can appreciate you being in shape.\n I've had/have an idea for you, and only YOU can do this. I would like to see you make a song/video which would be an answer to your dad's song ACHEY BREAKY HEART. If I were your dad, I would be TOUCHED! I don't know if you've had this in mind before, but I've had it in my mind just recently. What Say You?! And a VERY WARM CONGRADULATION for winning your very FIRST GRAMMY!",G7KNmW9a75Y,True
16,@danhartigan9529,2024-02-24T03:57:03Z,0,Great song just a shame we idolize cubby woman wearing tight clothes if she wore appropriate clothing this song would have done much better. What do the fox says exposed no s\nFat or skin and killed 😂,G7KNmW9a75Y,True
18,@shwetathapa-ly3td,2024-02-24T03:34:09Z,0,It gives strength listening to this\nYou remember what you got❤❤❤,G7KNmW9a75Y,True
22,@angeleyeszarai,2024-02-24T00:38:34Z,0,"I like Miley Cyrus. But her fame is SO manufactured..... it's almost sickening. Almost like they planned ""Hannah Montana"" becoming reality, since the 1st episode of that show. Much as I like her, she's too manufactured for me to fully enjoy. Same goes for Taylor Swift. And not to bring race into it, but them both being caucasian brings yet more suspicion to the topic. For me at least. \n\nStill talented though, either way. And I do like this song (granted I would've liked this song regardless who made it.... esp Lady Gaga who would've been a PERFECT fit for this song, no shade whatsoever. But it complements Miley's voice well too).",G7KNmW9a75Y,True


In [14]:
# Check for data in column 'text' that contain url substrings:
print(df['text'].str.contains(r'http[s]?\S+').value_counts())
df[df['text'].str.contains(r'http[s]?\S+') == True].head()

text
False    97434
True       480
Name: count, dtype: int64


,author,updated_at,like_count,text,video_id,public
0,@MileyCyrus,2023-01-13T00:39:35Z,158633,I can love me better than you can…. FLOWERS SINGLE + VIDEO OUT NOW. \n\nhttps://mileycyrus.lnk.to/Flowers,G7KNmW9a75Y,True
77,@fbisurveillancevan9320,2024-02-23T12:39:21Z,0,https://youtu.be/GvbQzRAi4wM?si=37If493RI52j6YwX,G7KNmW9a75Y,True
133,@amandarowland1866,2024-02-22T20:13:29Z,0,I am here because of @MRLBOYD https://youtu.be/XDpW_IoyVb4?si=7XWk-4yWAMlregpR,G7KNmW9a75Y,True
147,@loganfitnessstudio1354,2024-02-22T17:19:43Z,0,https://youtube.com/@loganfitnessstudio1354?si=SkxIarHIHf657VpR?sub_confirmation=1,G7KNmW9a75Y,True
176,@LofiWithTrend_01,2024-02-22T07:44:48Z,1,I am from India love my indian songs https://youtu.be/gMbBMckQmIQ?sub_confirmation=1,G7KNmW9a75Y,True


In [15]:
# Check for data in column 'text' that contain hashtags and emojis:
print(df['text'].str.contains(r'[#|#&].*?\s+').value_counts())
df[df['text'].str.contains(r'[#|#&].*?\s+') == True].head()

text
False    97009
True       905
Name: count, dtype: int64


,author,updated_at,like_count,text,video_id,public
189,@Archna12340,2024-02-22T02:29:20Z,0,pubg0#pubgshort#pubglite#bgmi#mummy shut #viral#subskairb#bgmishort#shortfeed#youtubeshorts#esports#,G7KNmW9a75Y,True
270,@prismgems,2024-02-21T01:26:40Z,1,"The algorithm gave me a list of the best songs of 2023, this was #1. Disappointment! They were all victim music of one sort or another. Nothing inspiring, nothing high energy. Not a single one of them was worth a crap. I guess they just reflect the world view of the artists. Sad. I feel sorry for the people who will live in the world they are creating with their music.",G7KNmW9a75Y,True
491,@sharonwarren4479,2024-02-18T17:32:57Z,3,"So good and empowering, Miley. Congrats on your Grammy.\n\n❤❤❤❤❤❤\n#MileyCyrus\n#1stGrammy",G7KNmW9a75Y,True
520,@name865name,2024-02-18T13:01:48Z,0,"HERE IS A REAL STORY!\r\nHow about Bringing Up the Entire NY State Court system as CORRUPT! \r\n1. Judges Erdogan OOZES CONTEMPT FOR THE LAW DAILY-besides being a smug PREENING Self-Impressed 'Psychopath.' 2. NY State Prosecutors seem to be FIXED ON LAWLESSNESS! WHERE ARE YOU?...PREENING BEFORE YOUR LOBBYISTS HANDLERS, BUYING EVERY SINGLE DECISION THAT APPEAR & COME-OUT-OF ALL NY STATE COURTS! 3. It is said, ""Good comes to those who Wait."" I am DISHEARTENED though HOPING/PINING/ANGUISHING that KARMA comes along… VIA AN ASPIRING LEGAL NY STATE ADVOCATE, TO ACTUALLY APPLY ‘JUSTICE-UNDER THE LAW.’",G7KNmW9a75Y,True
600,@tyrooneyfromdababooney7585,2024-02-17T23:04:45Z,0,"😜🤪🤪😜😝, still strap it up & hit it.",G7KNmW9a75Y,True


In [16]:
# Check for data in column 'text' that contain @ mentions:
print(df['text'].str.contains(r'@\S+').value_counts())
df[df['text'].str.contains(r'@\S+') == True].head()

text
False    97577
True       337
Name: count, dtype: int64


,author,updated_at,like_count,text,video_id,public
115,@lorenasmith7504,2024-02-23T02:37:04Z,0,Fuc@/-// AMAZING 🔥🔥🔥,G7KNmW9a75Y,True
133,@amandarowland1866,2024-02-22T20:13:29Z,0,I am here because of @MRLBOYD https://youtu.be/XDpW_IoyVb4?si=7XWk-4yWAMlregpR,G7KNmW9a75Y,True
134,@juliomaldonado-sabines9501,2024-02-22T20:07:58Z,0,i like your drees @Mileycyrus 🥰\n😇,G7KNmW9a75Y,True
147,@loganfitnessstudio1354,2024-02-22T17:19:43Z,0,https://youtube.com/@loganfitnessstudio1354?si=SkxIarHIHf657VpR?sub_confirmation=1,G7KNmW9a75Y,True
159,@yewinnaung6084,2024-02-22T15:15:42Z,0,Slay @miley,G7KNmW9a75Y,True


In [17]:
# Check for data in column 'text' that contain numbers:
print(df['text'].str.contains(r'[0-9]').value_counts())
df[df['text'].str.contains(r'[0-9]') == True].head()

text
False    81945
True     15969
Name: count, dtype: int64


,author,updated_at,like_count,text,video_id,public
5,@Ganesh-zs1iv,2024-02-24T08:07:13Z,0,It's Feb 2024 and still I am watching it,G7KNmW9a75Y,True
11,@user-wp4hh5bp1r,2024-02-24T04:45:02Z,0,At 2:07 u can see her ……….. I’m sorry if I’m being kinda rude and by the way the word will be I’m my comments,G7KNmW9a75Y,True
14,@user-cb4he9un6v,2024-02-24T04:08:42Z,0,Listening to this at home in my apartment while my boyfriend of 3 years is back at the motel drunk on 5ths of vodka a day.,G7KNmW9a75Y,True
22,@angeleyeszarai,2024-02-24T00:38:34Z,0,"I like Miley Cyrus. But her fame is SO manufactured..... it's almost sickening. Almost like they planned ""Hannah Montana"" becoming reality, since the 1st episode of that show. Much as I like her, she's too manufactured for me to fully enjoy. Same goes for Taylor Swift. And not to bring race into it, but them both being caucasian brings yet more suspicion to the topic. For me at least. \n\nStill talented though, either way. And I do like this song (granted I would've liked this song regardless who made it.... esp Lady Gaga who would've been a PERFECT fit for this song, no shade whatsoever. But it complements Miley's voice well too).",G7KNmW9a75Y,True
43,@dawooddiwan9994,2024-02-23T19:17:08Z,0,Anyone in 2024?,G7KNmW9a75Y,True


In [18]:
# Check for data in column 'tweet' that contain punctuations:
print(df['text'].str.contains(r'[^a-zA-Z0-9\s]').value_counts())
df[df['text'].str.contains(r'[^a-zA-Z0-9\s]') == True].head()

text
True     73651
False    24263
Name: count, dtype: int64


,author,updated_at,like_count,text,video_id,public
0,@MileyCyrus,2023-01-13T00:39:35Z,158633,I can love me better than you can…. FLOWERS SINGLE + VIDEO OUT NOW. \n\nhttps://mileycyrus.lnk.to/Flowers,G7KNmW9a75Y,True
3,@Royalchess1,2024-02-24T09:02:54Z,0,"Ms. Miley,\n I have alot of mixed emotions while watching your video. I could use many explanitives that you've heard before, but the words, the music and the video is something I've NEVER WITNESSED BEFORE. I MUST ADMIT, I've NEVER watched one of your videos before, but if I did, it's been awhile ago. I can tell you are in great shape, and I mean that in all honesty. Personally, I've been FORCED to suspend my VASA spa contract because of COVID, and I can't go mountain-biking (here in Utah) or play Olympic Table Tennis either because I have some operations to deal with, so I can appreciate you being in shape.\n I've had/have an idea for you, and only YOU can do this. I would like to see you make a song/video which would be an answer to your dad's song ACHEY BREAKY HEART. If I were your dad, I would be TOUCHED! I don't know if you've had this in mind before, but I've had it in my mind just recently. What Say You?! And a VERY WARM CONGRADULATION for winning your very FIRST GRAMMY!",G7KNmW9a75Y,True
4,@mikerooney7600,2024-02-24T09:01:41Z,0,I love this song! It makes every day better,G7KNmW9a75Y,True
5,@Ganesh-zs1iv,2024-02-24T08:07:13Z,0,It's Feb 2024 and still I am watching it,G7KNmW9a75Y,True
6,@ogtdkk5932,2024-02-24T07:06:47Z,0,"Si hay que idealizar alguna canción, esta es la correcta.....",G7KNmW9a75Y,True


In [19]:
# Convert strings in column 'text' to lowercase:
df['text'] = df['text'].str.lower()

In [20]:
# Convert contraction words in column 'text':

# Define contraction function:
def expand_contractions(text):
    return contractions.fix(text)

# Apply contraction function:
df['text'] = df['text'].apply(expand_contractions)

In [21]:
# Replace website urls in column 'text' with ' ':
df['text'] = df['text'].str.replace(pat='http[s]?\S+', repl=' ', regex=True)

In [22]:
# Function to remove emojis from text using a regex pattern
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001F900-\U0001F9FF"  # additional range for emojis
                               u"\U0001F100-\U0001F1FF"  # enclosed characters
                               u"\U0001F200-\U0001F2FF"  # enclosed ideographic supplement
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # geometric shapes extended
                               u"\U0001F800-\U0001F8FF"  # supplemental arrows-C
                               u"\U0001FA00-\U0001FA6F"  # chess symbols
                               u"\U0001FA70-\U0001FAFF"  # symbols and pictographs extended-A
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Remove emojis from the 'text' column in the DataFrame
df['text'] = df['text'].apply(remove_emojis)

In [23]:
# Replace @ mentions in column 'text' with ' ':
df['text'] = df['text'].str.replace(pat='@\S+', repl=' ', regex=True)

In [24]:
# Replace line of tab characters in column 'text' with ' ':
df['text'] = df['text'].str.replace(pat='\r|\t|\n', repl=' ', regex=True)

In [25]:
# Replace numbers in column 'text' with ' ':
df['text'] = df['text'].str.replace(pat='[0-9]', repl=' ', regex=True)

In [26]:
# Replace punctuations in column 'tweet' with ' ':
df['text'] = df['text'].str.replace(pat='[^a-zA-Z0-9\s]', repl=' ', regex=True)

In [27]:
# Replace empty whitespaces in column 'text' with ' ':
df['text'] = df['text'].str.replace(pat='\s+', repl=' ', regex=True)

In [28]:
# Remove trailing whitespaces in column 'text':
df['text'] = df['text'].str.strip()

In [29]:
# Remove unecessary whitespaces:
df['text'] = df['text'].str.split(' ').str.join(' ')

In [48]:
# Function to determine if the text is in English using the langdetect library
#!pip install langdetect
from langdetect import detect
from langdetect import DetectorFactory
DetectorFactory.seed = 0

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

# Apply the is_english function to create a new 'is_english' column in the DataFrame
df['is_english'] = df['text'].apply(is_english)

In [49]:
# Display the count of occurrences for each unique value in the 'is_english' column
df['is_english'].value_counts()

is_english
True     55829
False    42077
Name: count, dtype: int64

In [37]:
# export data frame to examine texts that have been classified as english and non-english:
df.to_csv('Data/youtube_comments_to_check_for_english.csv', index=False)

In [50]:
# Keep only the rows where 'is_english' column is True
df = df[df['is_english'] == True]

In [52]:
# Keep only the rows where 'like_count' column is less than or equal to 10000 as this are the official comments made by the artists of the music video
df = df[df['like_count'] <= 10000]

In [53]:
# Drop the 'is_english' column from the DataFrame
df = df.drop(columns=['is_english'])

In [54]:
# Reset index of dataframe
df = df.reset_index(drop=True)

In [55]:
# Check summary of dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55829 entries, 0 to 55828
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   author      55829 non-null  object
 1   updated_at  55829 non-null  object
 2   like_count  55829 non-null  int64 
 3   text        55829 non-null  object
 4   video_id    55829 non-null  object
 5   public      55829 non-null  bool  
dtypes: bool(1), int64(1), object(4)
memory usage: 2.2+ MB


In [56]:
# Create a boolean mask where True indicates rows with empty 'text' after stripping whitespace
empty_text_mask = df['text'].str.strip() == ''

# Apply the mask to the DataFrame to get rows with empty 'text'
empty_text_rows = df[empty_text_mask]

# Print the number of rows with empty 'text'
print(f"There are {len(empty_text_rows)} rows where 'text' contains no words.")

There are 0 rows where 'text' contains no words.



---

### 04 Save Cleaned Dataframe as CSV

In [57]:
# export final cleaned youtube comments data as csv:
df.to_csv('Data/youtube_comments_clean.csv', index=False)